## 라이브러리 설치

In [2]:
!pip install tk
!pip install pillow

You should consider upgrading via the 'C:\Users\PC\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\PC\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## 메인 게임 로직 코드 

In [117]:
import tkinter as tk
from tkinter import ttk
import random
from PIL import Image, ImageTk  # PIL 라이브러리 import


### 게임창, 플레이어 만들고, 아이템 만들기
- 경기 시작버튼 추가

In [118]:
# 게임창, 플레이어 만들기
def create_game_window():
    root = tk.Tk()
    root.title("누가누가 꼴찌할까?")
    
    # 게임창 크기
    canvas = tk.Canvas(root, width=1280, height=720)
    canvas.pack()

    # 여러 아이템 경로, 갯수 설정
    item_info = {
        "chestnut": {"path": "icon_img/chestnut.png", "count": 3},
        "banana": {"path": "icon_img/cobweb.png", "count": 2},
        "mushroom": {"path": "icon_img/poison.png", "count": 1}
    }
    
    # 이미지 참조를 root 객체에 저장(전역변수 느낌)
    root.images = []
    items = create_items(canvas, item_info, root.images)

    #플레이어 생성
    players = []
    for i in range(3):
        player = canvas.create_oval(50, 100+i*100, 70, 120+i*100, fill=f"#{random.randint(0, 0xFFFFFF):06x}")
        players.append(player)

    #경기 시작 버튼 생성
    start_button = ttk.Button(root, text="Start", command=lambda: start_race(canvas, players, start_button, items))
    start_button.pack()

    return root, canvas, players, items

### 캔버스에 아이템 로드하고 위치 랜덤 생성
- 추가해야하는 로직
1. 아이템 너무 몰려있지 않게하기

In [119]:

def create_items(canvas, item_info, images):
    items = []
    for item_type, info in item_info.items():
        image = Image.open(info["path"])
        image = image.resize((20, 20))
        item_photo = ImageTk.PhotoImage(image)
        images.append(item_photo)  # root.images에 참조 추가(전역변수 느낌)

        for _ in range(info["count"]):
            x = random.randint(100, 1200)
            y = random.randint(50, 670)
            item = canvas.create_image(x, y, image=item_photo)
            if item:  # 아이템이 성공적으로 생성되었는지 확인
                items.append((item, item_type))
            else:
                print(f"Failed to create item of type {item_type}")

    return items

### 경기 시작하는 버튼 (버튼 누르면 출발)
- 플레이어마다 순차적 호출
- canvas.after()로 0ms 후에 다시 실행되도록 예약
- 동시에 출발하는 것처럼 보임

In [120]:
def start_race(canvas, players, start_button, items):
    start_button.config(state='disabled')
    print(f"Starting race with {len(items)} items")  # 아이템 개수 출력

    for player in players:
        # 모든 플레이어 움직이게 하기
        canvas.after(0, lambda p=player: move_player(canvas, p, items))

### 플레이어를 움직이는 함수 
- 추가해야하는 로직 : 
1. 뒤로가기 함정 밟았을 시 내가 무조건 오른쪽으로만 이동하게 해놔서 계속 뒤로갔다가오른쪽가서또아이템밟아서 무한루프, 결국 ㄱㅓ기서돌고돌아 나아가질 못함. 
=> 뒤로 가기 함정 밟았을 때 앞으로 나아가게 하는 조건문 필요할듯

2. 아이템 밟고 제거하려면 remove 함수로 가능
=> 그런데 에러가 뜸(플레이어가 움직이지않고 정지함)

In [125]:

def move_player(canvas, player, items):
    print(f"Moving player {player}")  # 디버그 출력

    # 오른쪽으로 이동할 거리 (1에서 5 사이의 랜덤 값)
    dx = random.randint(1, 5)
    
    # 수직 방향 이동 결정 (-1: 위, 0: 직진, 1: 아래)
    vertical_direction = random.choice([-1, 0, 1])
    
    # 수직 이동 거리 설정 (위나 아래로 이동할 경우 1에서 3 사이의 랜덤 값)
    dy = vertical_direction * random.randint(5, 10) if vertical_direction != 0 else 0
    
    # 플레이어 이동
    # - dx: 항상 양수이므로 오른쪽으로만 이동
    # - dy: -3에서 3 사이의 값으로, 위, 직진, 또는 아래로 이동
    canvas.move(player, dx, dy)
    
    # 화면 경계 체크 및 조정
    player_coords = canvas.coords(player)
    if player_coords[1] < 0:  # 위쪽 경계를 벗어난 경우
        canvas.move(player, 0, -player_coords[1])
    elif player_coords[3] > 710:  # 아래쪽 경계를 벗어난 경우 (캔버스 높이가 400이라고 가정)
        canvas.move(player, 0, 710 - player_coords[3])
    
    # 아이템 충돌 체크
    for item,item_type in items[:]:
        # canvas 위젯에서 특정 아이템의 좌표를 가져오다
        item_coords = canvas.coords(item)
        # if item_coords : 이부분 없으면 게임로직 작동x
        # -- canvas.coords() 함수 자체가 때때로 오작동을 하기 때문
        # if item_coords and check_collision(player_coords, item_coords):
        if check_collision(player_coords, item_coords):

             # 아이템 종류에 따라 다른 이동 처리
            if item_type == "chestnut":
                canvas.move(player, -30, 0)  # 뒤로 3칸(30픽셀) 이동
            elif item_type == "banana":
                canvas.move(player, -30, 0)  # 뒤로 3칸(30픽셀) 이동
            elif item_type == "mushroom":
                canvas.move(player, +30, 0)  # 앞으로 3칸(30픽셀) 이동
            # canvas.delete(item)
            # items.remove(item)
            print(f"Player {player} got an item!")


    if not check_finish(canvas, player):
        print(f"Scheduling next move for player {player}")  # 디버그 출력
        canvas.after(100, lambda: move_player(canvas, player, items)) #0.01초 후 재실행(재움직임임)

        


### 아이템- 플레이어 충돌 (충돌기준 : 아이템 중앙좌표)

In [122]:

def check_collision(player_coords, item_coords):
    # 아래 코드 없으면 게임 로직 작동x
    # if not item_coords:
    #     return False
    # 플레이어의 중심 좌표 계산
    player_center_x = (player_coords[0] + player_coords[2]) / 2
    player_center_y = (player_coords[1] + player_coords[3]) / 2
    
    # 아이템의 중심 좌표
    item_center_x, item_center_y = item_coords
    
    # 플레이어와 아이템 중심 간의 거리 계산
    distance = ((player_center_x - item_center_x) ** 2 + (player_center_y - item_center_y) ** 2) ** 0.5
    
    # 충돌 판정 (플레이어의 반지름 + 아이템의 반지름)
    # 10으로 했을때
    # -- 분명 닿았는데 그냥 지나치는 경우가 있음 ㅠ_ㅠ 
    collision_distance = 10 + 10  # 플레이어와 아이템의 반지름이 각각 10픽셀이라고 가정
    
    return distance <= collision_distance



### 플레이어 도착점에 도착했나 구현

In [123]:
def check_finish(canvas, player):
    coords = canvas.coords(player)
    if coords[2] >= canvas.winfo_width():
        print(f"Player {player} 완주!")
        return True
    return False

### 게임 윈도우 생성 및 메인 루프 시작


In [124]:
# 게임 윈도우 생성 및 메인 루프 시작
root, canvas, players, items = create_game_window()
root.mainloop()

Starting race with 6 items
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Scheduling next move for player 8
Moving player 9
Scheduling next move for player 9
Moving player 7
Scheduling next move for player 7
Moving player 8
Schedul

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_11252\576442607.py", line 48, in <lambda>
    canvas.after(100, lambda: move_player(canvas, player, items)) #0.01초 후 재실행(재움직임임)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_11252\576442607.py", line 42, in move_player
    items.remove(item)
ValueError: list.remove(x): x not in list
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\PC\